In [1]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession, Row
spark = SparkSession.builder \
    .appName("PySparkShell") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .getOrCreate()

spark

In [3]:
df = spark.read.format("org.apache.spark.sql.cassandra").options(keyspace="live_feed", table="live_table").load()

In [4]:
df.show()

+--------------------+--------------------+---------------------+--------------------+---------+-----------+---------------+-------------+--------------------+
|            order_id|          created_at|discount_or_promotion|               email| latitude|  longitude|           name|paymentmethod|         total_price|
+--------------------+--------------------+---------------------+--------------------+---------+-----------+---------------+-------------+--------------------+
|eb262bc7-89b1-4ca...|1970-01-20 21:02:...|                 true|samantha34@exampl...|37.319435| -79.966355|    Linda Smith|  credit card|372.0000000000000...|
|032d2c64-a8ff-4ea...|1970-01-20 21:02:...|                 true|foxkevin@example.com|41.071368|-111.983774|    Donald Hill|         cash|900.0000000000000...|
|36b2b543-a7ca-4d6...|1970-01-20 21:02:...|                false|angelaparsons@exa...|37.319435| -79.966355|  Gloria Taylor|         cash|521.0000000000000...|
|5d061f1d-4988-4cf...|1970-01-20 21:02:.